In [1]:
import pyaudio
import os
import struct
import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import fft
import time
from tkinter import TclError
import noisereduce

# to display in separate Tk window
%matplotlib tk

# constants
CHUNK = 1024 * 2             # samples per frame
FORMAT = pyaudio.paInt16     # audio format (bytes per sample?)
CHANNELS = 1                 # single channel for microphone
RATE = 44100                 # samples per second

In [2]:
# create matplotlib figure and axes
fig, (ax1, ax2) = plt.subplots(2, figsize=(15, 7))

# pyaudio class instance
p = pyaudio.PyAudio()

# stream object to get data from microphone
stream = p.open(
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    input=True,
    output=True,
    frames_per_buffer=CHUNK
)

ax1.set_xlim(20, RATE / 2)
ax1.set_ylim(0, 3)
ax2.set_xlim(20, RATE / 2)
ax2.set_ylim(0, 3)



x = np.linspace(0, RATE, CHUNK)

origin, = ax1.semilogx(x, np.random.rand(CHUNK), '-', lw=2)
noise_reduced, = ax2.semilogx(x, np.random.rand(CHUNK), '-', lw=2)

print('stream started')

# for measuring frame rate
frame_count = 0
start_time = time.time()

while True:
    
    data = stream.read(CHUNK)  
    
    # convert data to integers, make np array, then offset it by 127
    data_int = struct.unpack(str(2 * CHUNK) + 'B', data)
    
    # create np array and offset by 128
    data_np = np.array(data_int, dtype='b')[::2] + 128
    y1f = fft(data_int)
    origin.set_ydata((np.abs(y1f[0:CHUNK])  / (128 * CHUNK))*10)
    
    noiseReduced = noisereduce.reduce_noise(y=data_np, sr= RATE)
    y2f = fft(noiseReduced)
    noise_reduced.set_ydata((np.abs(y2f[0:CHUNK])  / (128 * CHUNK))*10)
    
    # update figure canvas
    try:
        fig.canvas.draw()
        fig.canvas.flush_events()
        frame_count += 1
        
    except TclError:
        
        # calculate average frame rate
        frame_rate = frame_count / (time.time() - start_time)
        
        print('stream stopped')
        print('average frame rate = {:.0f} FPS'.format(frame_rate))
        break

stream started


: 

: 